In [5]:
from bs4 import BeautifulSoup as bs
import urllib
import requests
import pandas as pd
import re
import os
import sys
import json

In [26]:
# 네이버 API 로그인 후 link 정보 가져와 돌려주기
def get_Nav_client(encText, sPage):
    
    # url 생성
    url = "https://openapi.naver.com/v1/search/kin?query=" + urllib.parse.quote(encText) # json 결과
    url = url + "&start=" + str(sPage) + "&display=100"

    headers = {"X-Naver-Client-Id":"", 
               "X-Naver-Client-Secret":""}
    
    # 네이버 API 연결(로그인)
    res_content=requests.get(url, headers=headers)

    if res_content.status_code == 200:
        res_json = res_content.json()
    else:
        print("Error Code:" + str(res_json.status_code))
        sys.exit(0)

    df=pd.DataFrame(res_json['items']) # 기본 DataFrame 생성
    
    df_body=get_write(list(df['link'])) # 질문/답변 DataFrame 받아오기
    
    df=pd.concat([df, df_body], axis=1)
    #display(df.head(3))   
    return df

In [27]:
#link를 이용해 세부 내용 가져와 파일에 기록하기
def get_write(reg_link):
    
    body_lst = []
    
    for link in reg_link:
        kin_html = requests.get(link)

        if kin_html.status_code != 200 :
            print("Error Code:" + str(res_json.status_code))
            body_lst.append({"질문":"질문없음", "답변":"답변없음"})
            
        else:
            kin_soup = bs(kin_html.text, "html.parser")

            # 질문 내용
            try:
                kin_body=kin_soup.find('div', class_="c-heading__content").text.replace("\n", "").replace("\t", "")
            except:
                kin_body="질문없음"

            # 답변 내용
            ans = ""
            try:
                for kin_ans in kin_soup.find_all('div', class_="se-component se-text se-l-default"):
                    txt=re.sub("[^가-힣 ]","",kin_ans.text)
                    ans = ans + txt +"\n"
            except:
                ans="답변없음"

            body_lst.append({"질문":kin_body, "답변":ans})
            
    df_body=pd.DataFrame(body_lst)
    #display(df_body.head(3))
    return df_body


In [28]:
encText = input("검색어 입력:")

df_navkin=pd.DataFrame()   # 최종 데이터 저장

for i in range(1, 101,100):

    df = get_Nav_client(encText, i)
    df_navkin=pd.concat([df_navkin, df])

df_navkin

검색어 입력:제주도


,title,link,description,질문,답변
0,<b>제주도</b> 신혼여행 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=9&...,... 신혼여행을 5월1일날 <b>제주도</b>로 가게되었는데 그때의 <b>제주도<...,4월에 결혼식을 앞둔 예비신랑입니다. 신혼여행을 5월1일날 제주도로 가게되었는데그때...,건강한 사람이라면 반팔에 바람막이 얇은 옷 하나 들고 가세요접으면 주먹만한 ...
1,<b>제주도</b> 3박4일 여행코스,https://kin.naver.com/qna/detail.naver?d1id=9&...,2월 26일 3박4일로 여자친구랑 <b>제주도</b> 여행을 가려고 하는데 <b>제...,2월 26일 3박4일로 여자친구랑 제주도 여행을 가려고 하는데 제주도를 많이 가보지...,여자친구랑 제주도 여행을 가려고 하는군요아래에 박일로 해서 여행코스를 짜보았...
2,<b>제주도</b> 호텔 숙박,https://kin.naver.com/qna/detail.naver?d1id=9&...,... <b>제주도</b>로 혼자 2박3일동안 갔다올생각인데 <b>제주도</b> 호...,제가 15살인데 제주도로 혼자 2박3일동안 갔다올생각인데 제주도 호텔에서 미성년자가...,혼자서 제주도로 여행을 갔다 올 생각이군요미성년자일 경우 숙소는 문제가 될 ...
3,"<b>제주도</b> 숙소,맛집 추천",https://kin.naver.com/qna/detail.naver?d1id=9&...,<b>제주도</b>에 있는 1박에 30만원 이하인 리조트 추천해주세요. 객실도 좋고...,제주도에 있는 1박에 30만원 이하인 리조트 추천해주세요.객실도 좋고 조식도 괞잖은...,리조트는 켄싱턴이 좋았구요맛집은 요기 괜찮았구요 \n 요기는 배달...
4,<b>제주도</b> 렌트카,https://kin.naver.com/qna/detail.naver?d1id=12...,이제 갓 면허땃는데 <b>제주도</b>에서 렌트카 빌릴수엤나요 <b>제주도</b>...,이제 갓 면허땃는데 제주도에서 렌트카 빌릴수엤나요,보통은 면허 취득 후 년 이상 되셔야 합니다 제주 렌트카 최저가 가격비교 방...
...,...,...,...,...,...
95,<b>제주도</b>렌트카 가격비교,https://kin.naver.com/qna/detail.naver?d1id=8&...,가족들과 <b>제주도</b> 가려고 <b>제주도</b>렌트카 알아보는데요. <b>제...,가족들과 제주도 가려고 제주도렌트카 알아보는데요.제주도렌트카 업체들 마다 가격이 달...,안녕하세요 제주도렌트카 관련 인플루언서 입니다 대부분 사람들이 코로나 때문에...
96,<b>제주도</b> 2박3일 여행 코스,https://kin.naver.com/qna/detail.naver?d1id=12...,안녕하세용 10월9일~10월11일 2박3일 <b>제주도</b> 여행을 갈려고 합니다...,"안녕하세용10월9일~10월11일 2박3일 제주도 여행을 갈려고 합니다8세, 6세 아...",제주도 박일 여행 코스답변드릴게요질문자님께서 적어주신 내용을 참고로 제주도 ...
97,<b>제주도</b> 갈만한곳,https://kin.naver.com/qna/detail.naver?d1id=12...,<b>제주도</b> 서귀포쪽으로 놀만한곳있을까요? <b>제주도</b> 약간 청소년들...,제주도 서귀포쪽으로 놀만한곳있을까요?제주도 약간 청소년들이 재미를 느낄만한곳예를 들...,제주도로 여행을 가는군요아래에 제주도에서 즐길 수 있는 액티비티한 체험을 정...
98,<b>제주도</b> 맛집,https://kin.naver.com/qna/detail.naver?d1id=12...,<b>제주도</b>로 7월 21일쯤에 놀러 가는데 <b>제주도</b> 맛집 좀 추천...,제주도로 7월 21일쯤에 놀러 가는데 제주도 맛집 좀 추천해주세요ㅜㅜ,제주 맛집 순위로 보여주는곳 링크 첨부해드릴게요순위로 정리되어 있는 사이트에...


In [13]:
df_navkin.to_csv('c:/pydata/' + encText + "_지식인.csv", index = False)

""
